In [1]:
# Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
!apt-get update  
!apt-get install libffi-dev  # Install libffi-dev
!pip install --upgrade pip  # Ensure pip is up to date
!pip install pysus # Exa
!pip install gdown
!pip install pylance

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
pip install pylance

Note: you may need to restart the kernel to use updated packages.


In [1]:
# importar bibliotecas para carga dos dados
import pysus
import os
import sidrapy
import requests # para carga de endereços web - api
from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

In [2]:
import pandas as pd

In [12]:

dobr2010 = sim.get_files("CID10", uf=["BR"], year=[2010])
dobr2010

NameError: name 'sim' is not defined

In [9]:
translate_variables_SIM

<function pysus.preprocessing.decoders.translate_variables_SIM(dataframe: pandas.core.frame.DataFrame, age_unit: str = 'Y', age_classes=None, classify_args: dict = {}, classify_cid10_chapters=False, geocode_dv=True, nan_marker=None, category_columns=True)>

In [7]:
cid10 = get_CID10_table()
cid10

2024-12-26 16:45:33.102 | DEBUG    | pysus.online_data.SIM:get_CID10_table:116 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2024-12-26 16:45:33.139 | DEBUG    | pysus.online_data.SIM:get_CID10_table:120 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2024-12-26 16:45:33.740 | INFO     | pysus.online_data.SIM:get_CID10_table:146 - Data stored as parquet at /home/codespace/pysus/SIM_CID10_.parquet
2024-12-26 16:45:33.743 | DEBUG    | pysus.online_data.SIM:get_CID10_table:149 - CID10.DBF removed


,CID10,OPC,CAT,SUBCAT,DESCR,RESTRSEXO
0,A00,,S,N,A00 Colera,5
1,A000,,N,S,A00.0 Colera dev Vibrio cholerae 01 biot cholerae,5
2,A001,,N,S,A00.1 Colera dev Vibrio cholerae 01 biot El Tor,5
3,A009,,N,S,A00.9 Colera NE,5
4,A01,,S,N,A01 Febres tifoide e paratifoide,5
...,...,...,...,...,...,...
14252,Z926,,N,S,Z92.6 História pessoal de quimioterapia para d...,5
14253,U070,,N,S,U07.0 Doença por cigarro eletrônico,5
14254,U071,,N,S,U07.1 Infecção pelo novo Coronavírus (COVID-19),5
14255,U072,,N,S,"U07.2 COVID-19, vírus não identificado",5


In [3]:
# Testar código de município

mun_censo = pd.read_csv('files_clean/censo_pop_clean.csv')
dobr = pd.read_csv('files_clean/dobr_clean.csv')

/tmp/ipykernel_3277/3092962000.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  mun_censo = pd.read_csv('files_clean/censo_pop_clean.csv')


In [49]:
sim_cadmun = pd.read_parquet('downloads_sim_original/SIM_CADMUN_.parquet')
sim_cadmun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5652 entries, 0 to 5651
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MUNCOD      5652 non-null   object 
 1   MUNCODDV    5652 non-null   object 
 2   SITUACAO    5652 non-null   object 
 3   MUNSINP     5652 non-null   object 
 4   MUNSIAFI    5652 non-null   object 
 5   MUNNOME     5652 non-null   object 
 6   MUNNOMEX    5652 non-null   object 
 7   OBSERV      5652 non-null   object 
 8   MUNSINON    5652 non-null   object 
 9   MUNSINONDV  5652 non-null   object 
 10  AMAZONIA    5652 non-null   object 
 11  FRONTEIRA   5652 non-null   object 
 12  CAPITAL     5652 non-null   object 
 13  UFCOD       5652 non-null   object 
 14  MESOCOD     5652 non-null   object 
 15  MICROCOD    5652 non-null   object 
 16  MSAUDCOD    5652 non-null   object 
 17  RSAUDCOD    5652 non-null   object 
 18  CSAUDCOD    5652 non-null   object 
 19  RMETRCOD    5652 non-null  

In [55]:

sim_cadmun[sim_cadmun['MUNCOD'] == '150475']

,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,...,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA


In [59]:
# Trazer registro de sim_cadmun que tenha algum parte de MUNNOME com 'Natal'
sim_cadmun[sim_cadmun['MUNNOME'].isin(['dos Campos'])]          


,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,...,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA


In [44]:
mun_censo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111420 entries, 0 to 111419
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Ano          111419 non-null  object 
 1   mun_cod      111400 non-null  float64
 2   mun_nome     111400 non-null  object 
 3   cor          111400 non-null  float64
 4   Sexo         111400 non-null  float64
 5   pop_total    111420 non-null  int64  
 6   pop_under5   111420 non-null  int64  
 7   pop_under74  111420 non-null  int64  
 8   UF           111400 non-null  object 
 9   CODMUN_SIM   111280 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 8.5+ MB


In [12]:
mun_sim = pd.read_parquet('downloads_sim_original/SIM_CADMUN_.parquet')

In [47]:
mun_sim[['MUNCODDV', 'MUNCOD','CAPITAL']].head()

,MUNCODDV,MUNCOD,CAPITAL
0,0000000,000000,N
1,1100000,110000,N
2,1100015,110001,N
3,1100023,110002,N
4,1100031,110003,N


In [24]:
# Mostrar os registros com CODMIN_SIM = NaN
mun_censo[mun_censo.CODMUN_SIM.isna()]


,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF,CODMUN_SIM
2240,2010,1504752.0,Mojuí dos Campos,1.0,1.0,0,0,0,PA,NaN
2241,2010,1504752.0,Mojuí dos Campos,1.0,2.0,0,0,0,PA,NaN
2242,2010,1504752.0,Mojuí dos Campos,2.0,1.0,0,0,0,PA,NaN
2243,2010,1504752.0,Mojuí dos Campos,2.0,2.0,0,0,0,PA,NaN
2244,2010,1504752.0,Mojuí dos Campos,3.0,1.0,0,0,0,PA,NaN
...,...,...,...,...,...,...,...,...,...,...
111415,0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
111416,X,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
111417,..,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
111418,...,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN


In [53]:
# Trazer todos os registros que para o mun_censo.mun_cod começam com '150475'
mun_censo[mun_censo.mun_cod.astype(str).str.startswith('150475')]

,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF,CODMUN_SIM
2240,2010,1504752.0,Mojuí dos Campos,1.0,1.0,0,0,0,PA,NaN
2241,2010,1504752.0,Mojuí dos Campos,1.0,2.0,0,0,0,PA,NaN
2242,2010,1504752.0,Mojuí dos Campos,2.0,1.0,0,0,0,PA,NaN
2243,2010,1504752.0,Mojuí dos Campos,2.0,2.0,0,0,0,PA,NaN
2244,2010,1504752.0,Mojuí dos Campos,3.0,1.0,0,0,0,PA,NaN
2245,2010,1504752.0,Mojuí dos Campos,3.0,2.0,0,0,0,PA,NaN
2246,2010,1504752.0,Mojuí dos Campos,4.0,1.0,0,0,0,PA,NaN
2247,2010,1504752.0,Mojuí dos Campos,4.0,2.0,0,0,0,PA,NaN
2248,2010,1504752.0,Mojuí dos Campos,5.0,1.0,0,0,0,PA,NaN
2249,2010,1504752.0,Mojuí dos Campos,5.0,2.0,0,0,0,PA,NaN


In [10]:
dobr.CODMUNRES

0         150330
1         150010
2         150330
3         150020
4         150020
           ...  
874016    210312
874017    310930
874018    150150
874019    150080
874020    150140
Name: CODMUNRES, Length: 874021, dtype: int64

In [25]:
dobr[dobr.CODMUNRES == 150475]

,CAUSABAS,CODMUNRES,SEXO,RACACOR,OBITO_FAIXA_ETARIA,ANO
79982,B342,150475,2,4,under74,2022
80350,K639,150475,1,1,under74,2022
85015,I219,150475,1,4,under74,2022
113574,R99,150475,1,4,under74,2022
113575,V445,150475,1,4,under74,2022
118071,P200,150475,1,4,under74,2022
141572,V99,150475,1,4,under74,2022
144335,I64,150475,2,4,under74,2022
145466,I219,150475,1,2,under74,2022
150928,I461,150475,1,1,under74,2022


In [15]:
mun_sim.MUNCOD

0       000000
1       110000
2       110001
3       110002
4       110003
         ...  
5647    522208
5648    522210
5649    522220
5650    522230
5651    530010
Name: MUNCOD, Length: 5652, dtype: object

In [31]:
mun_censo[mun_censo['CODMUN_SIM'] == 150475]

,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF,CODMUN_SIM


In [51]:
mun_censo[mun_censo['mun_cod'] == '150475']

,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF,CODMUN_SIM


In [62]:
# procurar 2 códigos de município
mun_censo[mun_censo['CODMUN_SIM'].isin(['150475'])]

,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF,CODMUN_SIM


In [ ]:
mun_censo[mun_censo['CODMUN_SIM'].isin(['150475'])]

In [60]:
# procurar 2 códigos de município
sim_cadmun[sim_cadmun['MUNCOD'].isin(['110001', '150475'])]

,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,...,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA
2,110001,1100015,ATIVO,26016,0033,Alta Floresta D'Oeste,ALTA FLORESTA D'OESTE,,,,...,11900,1190,1190,1986,,,-11.929,-61.996,350.0,7066.702


In [66]:
sim_cadmun[sim_cadmun['MUNNOME'].str.contains('dos Campos')]

,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,...,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA
1754,270860,2708600,ATIVO,02086,2871,SÆo Miguel dos Campos,SAO MIGUEL DOS CAMPOS,,,,...,27003,2790,2790,1832,,,-9.781,-36.094,12.0,360.846
2200,292930,2929305,ATIVO,04293,3885,SÆo Gonalo dos Campos,SAO GONCALO DOS CAMPOS,,,,...,29044,2990,2990,1884,,,-12.433,-38.967,234.0,293.989
3843,354990,3549904,ATIVO,21500,7099,SÆo Jos dos Campos,SAO JOSE DOS CAMPOS,,,,...,35900,3590,3512,1767,,,-23.179,-45.887,600.0,1099.613
4882,431237,4312377,ATIVO,19455,1000,Monte Alegre dos Campos,MONTE ALEGRE DOS CAMPOS,,,,...,43900,4390,4390,1997,,,-28.683,-50.783,926.0,549.740


In [ ]:
# trazer registro de sim_cadmun que no campo MUNNOME contenha palavra 'Floresta'


In [64]:
# procurar 2 códigos de município
sim_cadmun[sim_cadmun['MUNCODDV'].isin(['1100010', 1504752])]

,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,...,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA


In [16]:
mun_sim.MUNCODDV

0       0000000
1       1100000
2       1100015
3       1100023
4       1100031
         ...   
5647    5222088
5648    5222104
5649    5222203
5650    5222302
5651    5300108
Name: MUNCODDV, Length: 5652, dtype: object

## Testes do dia 03/01

In [15]:
# Carga de arquivo de População do Censo
teste_uf_raw = pd.read_parquet('downloads_outros_v2/censo_pop_raw.parquet')
teste_uf_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11194 entries, 0 to 11193
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Ano                               11194 non-null  object
 1   Nível                             11194 non-null  object
 2   Cód.                              11194 non-null  object
 3   Unidade da Federação e Município  11194 non-null  object
 4   Cor ou raça                       11194 non-null  object
 5   Sexo                              11194 non-null  object
 6   Total                             11194 non-null  object
 7   75 a 79 anos                      11194 non-null  object
 8   80 a 84 anos                      11194 non-null  object
 9   85 a 89 anos                      11194 non-null  object
 10  90 a 94 anos                      11194 non-null  object
 11  95 a 99 anos                      11194 non-null  object
 12  100 anos ou mais  

In [16]:
teste_uf_raw.Nível.unique()

array(['UF', 'MU'], dtype=object)

In [19]:
teste_uf_ = teste_uf_raw[teste_uf_raw['Nível'] == 'UF']
teste_uf_[['Ano','Unidade da Federação e Município','Total']].head(10)


,Ano,Unidade da Federação e Município,Total
0,2010,Rondônia,1562409
1,2010,Acre,733559
2,2010,Amazonas,3483985
3,2010,Roraima,450479
4,2010,Pará,7581051
5,2010,Amapá,669526
6,2010,Tocantins,1383445
7,2010,Maranhão,6574789
8,2010,Piauí,3118360
9,2010,Ceará,8452381


In [ ]:
# teste_uf_['Cód.'].astype(str).str[:2]
teste_uf_['Total'] = teste_uf_['Total'].astype(int)
teste_uf = teste_uf_.groupby('uf_co')['Total'].sum()
teste_uf.head(20)

In [8]:
# Colunas para transformar em inteiro
cols_age = ['Total','0 a 4 anos', '75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
# para todas as colunas identificar substituir '-' por 0
teste_uf_ = censo_pop.copy()
teste_uf_[cols_age] = teste_uf_[cols_age].replace('-',0)

# transformar todas as colunas definidas para formato integer
teste_uf_[cols_age] = teste_uf_[cols_age].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
teste_uf_.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111420 entries, 0 to 111419
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        111420 non-null  object
 1   Ano               111419 non-null  object
 2   Nível             111407 non-null  object
 3   Cód.              111400 non-null  object
 4   Município         111400 non-null  object
 5   Cor ou raça       111400 non-null  object
 6   Sexo              111400 non-null  object
 7   Total             111420 non-null  int64 
 8   0 a 4 anos        111420 non-null  int64 
 9   75 a 79 anos      111420 non-null  int64 
 10  80 a 84 anos      111420 non-null  int64 
 11  85 a 89 anos      111420 non-null  int64 
 12  90 a 94 anos      111420 non-null  int64 
 13  95 a 99 anos      111420 non-null  int64 
 14  100 anos ou mais  111420 non-null  int64 
dtypes: int64(8), object(7)
memory usage: 12.8+ MB


In [20]:
teste_uf_ = censo_pop_clean03[censo_pop_clean03['Nível'] == 'MU']
teste_uf_.head()

NameError: name 'censo_pop_clean03' is not defined

In [2]:
import pandas as pd

def save_file(df, save_file_ent, save_type, versao_label, save_format):
    """
    Salva um DataFrame em um arquivo, com base nas configurações fornecidas.

    Args:
        df: O DataFrame a ser salvo.
        save_file_ent: Nome base do arquivo.
        save_type: Tipo de salvamento (raw, temp, final).
        versao_label: Rótulo da versão.
        save_format: Formato do arquivo (parquet, csv, excel).
    """

    # Define o diretório de saída com base no tipo de salvamento
    directories = {'raw': 'files_in/', 'temp': 'files_temp/', 'final': 'files_out/'}
    directory = directories[save_type]

    # Extrai a versão do rótulo
    save_vers = versao_label.split('__')[0]

    # Extrai a extensão do nome do arquivo base
    save_file_exit = save_file_ent.split('_')[-1]

    # Constrói o nome completo do arquivo
    filename = f"{directory}{save_vers}__{save_file_ent}.{save_format}"

    # Seleciona a função de salvamento de acordo com o formato
    save_functions = {'parquet': df.to_parquet, 'csv': df.to_csv, 'excel': df.to_excel}
    save_function = save_functions[save_format]

    # Salva o arquivo
    save_function(filename, index=False)



In [ ]:
# Exemplo de uso:
df_temp01 = pd.DataFrame({'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']})


In [ ]:


def save_file(df, save_type, save_format):
    save_file_ent = input("df_temp01")
    versao_label = input("AvoidMort_v4_BR_CID_mun")
    # ... resto da função ...


save_file(df, 'dobr_evit_merged_temp01', 'temp', 'AvoidMort_v4__BR_CID_mun', 'parquet')


In [ ]:

def save_file(df, save_file_ent='meu_arquivo', versao_label='v1.0', save_type='temp', save_format='parquet'):
    # ... resto da função ...